In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding
from keras.layers import Input
from keras.layers.merge import Concatenate
from tensorflow.keras.utils import to_categorical
from keras.models import Model
import matplotlib.pyplot as plt
import re
import string
import numpy as np
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm_notebook

In [71]:
def read_text_file(file_path):
    with open(file_path, 'r',encoding="UTF-8") as f:
        return f.read()
    
training_text = ""
# iterate through all files
max_file = 3
count = 0
for file in os.listdir("../Data/"):
    count+=1
    if count>max_file:
        break
    print(file)
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = "../Data/"+file
        training_text+=read_text_file(file_path)

Book 1 - The Philosopher's Stone.txt
Book 2 - The Chamber of Secrets.txt
Book 3 - The Prisoner of Azkaban.txt


In [72]:
len(training_text)

1683115

In [73]:
cleaned = re.sub(r'\W+', ' ', training_text).lower()
tokens = word_tokenize(cleaned)

In [74]:
train_len = 4
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

In [75]:
text_sequences[0:10]

[['the', 'boy', 'who', 'lived'],
 ['boy', 'who', 'lived', 'mr'],
 ['who', 'lived', 'mr', 'and'],
 ['lived', 'mr', 'and', 'mrs'],
 ['mr', 'and', 'mrs', 'dursley'],
 ['and', 'mrs', 'dursley', 'of'],
 ['mrs', 'dursley', 'of', 'number'],
 ['dursley', 'of', 'number', 'four'],
 ['of', 'number', 'four', 'privet'],
 ['number', 'four', 'privet', 'drive']]

In [76]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
train_sequences = tokenizer.texts_to_sequences(text_sequences)
print('Found %s unique tokens.' % len(tokenizer.word_counts))
vocab =  len(tokenizer.word_counts)+1

Found 11451 unique tokens.


In [77]:
n_sequences = np.empty([len(train_sequences),train_len], dtype='int32')
for i in range(len(train_sequences)):
    n_sequences[i] = train_sequences[i]

x = n_sequences[:,:-1]
y = n_sequences[:,-1]
#y_cat = to_categorical(y, num_classes=vocab)
#seq_len = train_inputs.shape[1]
#train_inputs.shape

In [78]:
y = to_categorical(y,num_classes=vocab)

In [82]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocab,10, input_length=3))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# compile network
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(x,y_cat,epochs=,verbose=1)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 3, 10)             114520    
_________________________________________________________________
lstm_18 (LSTM)               (None, 3, 50)             12200     
_________________________________________________________________
lstm_19 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_18 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_19 (Dense)             (None, 11452)             584052    
Total params: 733,522
Trainable params: 733,522
Non-trainable params: 0
_________________________________________________________________


In [83]:
history = model.fit(x, y,validation_split=0.05,batch_size=128, epochs=50)

Epoch 1/50
2189/2189 [==============================] - 110s 48ms/step - loss: 6.6268 - accuracy: 0.0516 - val_loss: 6.1287 - val_accuracy: 0.0872
Epoch 2/50
2189/2189 [==============================] - 3708s 2s/step - loss: 5.9833 - accuracy: 0.1054 - val_loss: 5.7400 - val_accuracy: 0.1325
Epoch 3/50
2189/2189 [==============================] - 110s 50ms/step - loss: 5.6396 - accuracy: 0.1335 - val_loss: 5.5522 - val_accuracy: 0.1505
Epoch 4/50
2189/2189 [==============================] - 106s 48ms/step - loss: 5.4103 - accuracy: 0.1469 - val_loss: 5.4676 - val_accuracy: 0.1507
Epoch 5/50
2189/2189 [==============================] - 115s 52ms/step - loss: 5.2418 - accuracy: 0.1557 - val_loss: 5.4300 - val_accuracy: 0.1615
Epoch 6/50
2189/2189 [==============================] - 121s 55ms/step - loss: 5.1122 - accuracy: 0.1614 - val_loss: 5.4223 - val_accuracy: 0.1560
Epoch 7/50
2189/2189 [==============================] - 113s 52ms/step - loss: 5.0086 - accuracy: 0.1662 - val_loss: 5.

KeyboardInterrupt: 

In [84]:
predicts = model.predict([x_test])
predict_logits = predicts.argmax(axis=1)

NameError: name 'x_test' is not defined